In [1]:
import pandas as pd

## Trainingsdaten laden

In [2]:
train_df = pd.read_csv("../data/train.csv")
train_df.head()

,transactionId,basket,customerType,totalAmount,returnLabel
0,9534310106,"[4, 3, 4]",new,252.0,1
1,7202594767,"[4, 2, 0, 2, 5]",existing,70.0,0
2,2737331698,[5],existing,84.0,0
3,4868011733,"[1, 4, 2, 4]",existing,116.0,0
4,7622406570,"[2, 5, 3, 2, 3, 2, 0]",existing,378.0,0


## Feature Engineering

In [3]:
def create_features(data_frame):
    data_frame = pd.get_dummies(data_frame, columns=["customerType"], dtype=int, drop_first=True)
    data_frame["orderedBooks"] = data_frame["basket"].apply(lambda x: sum(c.isdigit() for c in x))
    return data_frame

train_df = create_features(train_df)
train_df.head()

,transactionId,basket,totalAmount,returnLabel,customerType_new,orderedBooks
0,9534310106,"[4, 3, 4]",252.0,1,1,3
1,7202594767,"[4, 2, 0, 2, 5]",70.0,0,0,5
2,2737331698,[5],84.0,0,0,1
3,4868011733,"[1, 4, 2, 4]",116.0,0,0,4
4,7622406570,"[2, 5, 3, 2, 3, 2, 0]",378.0,0,0,7


## Daten skalieren

In [4]:
from sklearn.preprocessing import StandardScaler

features = ["totalAmount", "orderedBooks"]
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
train_df.head()

,transactionId,basket,totalAmount,returnLabel,customerType_new,orderedBooks
0,9534310106,"[4, 3, 4]",-0.032823,1,1,-0.607275
1,7202594767,"[4, 2, 0, 2, 5]",-0.879459,0,0,0.133204
2,2737331698,[5],-0.814334,0,0,-1.347754
3,4868011733,"[1, 4, 2, 4]",-0.665475,0,0,-0.237036
4,7622406570,"[2, 5, 3, 2, 3, 2, 0]",0.553309,0,0,0.873682


In [5]:
X_train = train_df.drop(columns=["returnLabel", "transactionId", "basket"])
y_train = train_df["returnLabel"]

In [6]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, 20, 100],
    "min_samples_split": [2, 5, 10]
}

In [7]:
from sklearn.model_selection import ParameterGrid

grid = list(ParameterGrid(param_grid))
grid[:5]

[{'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50},
 {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100},
 {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200},
 {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50},
 {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}]

## Modell auf Testdaten anwenden

In [8]:
test_df = pd.read_csv("../data/test.csv")
test_df.head()

,transactionId,basket,customerType,totalAmount,returnLabel
0,4132523932,"[4, 3, 4, 3, 2, 3]",existing,366.0,1
1,8998574539,"[3, 4, 4, 3, 5]",existing,85.0,0
2,9346688547,"[1, 1, 2, 2, 4, 4, 3, 1, 1, 0, 3]",existing,275.0,0
3,4533897707,"[3, 2, 2, 1, 5, 1, 1, 0]",existing,528.0,0
4,3334800500,"[4, 2, 3, 5, 2, 5, 1]",existing,287.0,0


In [9]:
test_df = create_features(test_df)
test_df.head()

,transactionId,basket,totalAmount,returnLabel,customerType_new,orderedBooks
0,4132523932,"[4, 3, 4, 3, 2, 3]",366.0,1,0,6
1,8998574539,"[3, 4, 4, 3, 5]",85.0,0,0,5
2,9346688547,"[1, 1, 2, 2, 4, 4, 3, 1, 1, 0, 3]",275.0,0,0,11
3,4533897707,"[3, 2, 2, 1, 5, 1, 1, 0]",528.0,0,0,8
4,3334800500,"[4, 2, 3, 5, 2, 5, 1]",287.0,0,0,7


In [10]:
features = ["totalAmount", "orderedBooks"] 
test_df[features] = scaler.transform(test_df[features])
test_df.head()

,transactionId,basket,totalAmount,returnLabel,customerType_new,orderedBooks
0,4132523932,"[4, 3, 4, 3, 2, 3]",0.497487,1,0,0.503443
1,8998574539,"[3, 4, 4, 3, 5]",-0.809682,0,0,0.133204
2,9346688547,"[1, 1, 2, 2, 4, 4, 3, 1, 1, 0, 3]",0.074169,0,0,2.354640
3,4533897707,"[3, 2, 2, 1, 5, 1, 1, 0]",1.251086,0,0,1.243922
4,3334800500,"[4, 2, 3, 5, 2, 5, 1]",0.129991,0,0,0.873682


In [11]:
X_test = test_df.drop(["transactionId", "returnLabel", "basket"], axis=1)
y_test = test_df["returnLabel"]

## MLFlow Part

In [12]:
import mlflow
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score
from mlflow.models import infer_signature
from mlflow import MlflowClient

mlflow.set_tracking_uri(uri="http://localhost:8080")
mlflow.set_experiment("MLflow Return Data HyperParamTuning")

client = MlflowClient()

for params in grid:
    with mlflow.start_run():

        mlflow.set_tag("Training Info", "Basic RF model for return data")
        
        rf = RandomForestClassifier(**params)
        rf.fit(X_train, y_train)

        predictions = rf.predict(X_test)

        accuracy = accuracy_score(y_test, predictions)
        precision = precision_score(y_test, predictions)
        recall = recall_score(y_test, predictions)
        
        # Log theparameters and tags
        mlflow.log_params(params)
        
        # Log the loss metric
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)

print("done.")

2025/11/03 16:19:42 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Return Data HyperParamTuning' does not exist. Creating a new experiment.


🏃 View run beautiful-cub-570 at: http://localhost:8080/#/experiments/220418263317249410/runs/6296d22c130447b1a7078b13adb5dd49
🧪 View experiment at: http://localhost:8080/#/experiments/220418263317249410
🏃 View run enthused-zebra-93 at: http://localhost:8080/#/experiments/220418263317249410/runs/401d7b66ebf1456f9f73b0392079f481
🧪 View experiment at: http://localhost:8080/#/experiments/220418263317249410
🏃 View run unleashed-seal-71 at: http://localhost:8080/#/experiments/220418263317249410/runs/aed135714da844598bdbccc7c11dd82f
🧪 View experiment at: http://localhost:8080/#/experiments/220418263317249410
🏃 View run silent-ray-517 at: http://localhost:8080/#/experiments/220418263317249410/runs/2a2eaa3d86804bf69e472c73eb0b9288
🧪 View experiment at: http://localhost:8080/#/experiments/220418263317249410
🏃 View run unequaled-mole-260 at: http://localhost:8080/#/experiments/220418263317249410/runs/60571e2a9a854d5e951ececb984c536d
🧪 View experiment at: http://localhost:8080/#/experiments/220418